In [2]:
import requests
from bs4 import BeautifulSoup
import time

import pandas as pd
import numpy as np

In [3]:
def get_text_from_genius_of(author, title):
    author = author.replace(" ", "-")
    title = title.replace(" ", "-")
    URL = "https://genius.com/" + author + "-" + title + "-lyrics"
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    
    lyrics = ""
    
    lyrics_containers = soup.find_all("div", {"data-lyrics-container" : "true"})
    
    if len(lyrics_containers) == 0:
        return None
    
    for lyrics_container in lyrics_containers:
        for br in lyrics_container.find_all("br"):
            br.replace_with("\n")
        lyrics += lyrics_container.text
    
    next_brackets = lyrics.find("[")
    while next_brackets != -1:
        lyrics = lyrics[:next_brackets] + lyrics[lyrics.find("]")+1:]
        next_brackets = lyrics.find("[")
    
    lyrics = lyrics.replace("\n\n", "\n").replace("\n", " ")
    return lyrics

In [52]:
songsdata_df = pd.read_csv('batch1.csv')
songsdata_df = songsdata_df[['track', 'artist', 'genre', 'valence_tags', 'arousal_tags']]
display(songsdata_df)

,track,artist,genre,valence_tags,arousal_tags
0,Every Soul Entombed,Skáphe,black metal,3.960000,3.180000
1,Push Your Tush,Jessica Simpson,pop,6.216667,5.646667
2,Vanilla Twilight,Owl City,electronic,7.125472,4.769623
3,Fly With Me,Relaxing Dreams,electronic,6.840000,5.000000
4,The Vampire Club,Voltaire,dark cabaret,3.398125,3.245625
...,...,...,...,...,...
7495,Ride A White Swan,T. Rex,glam rock,7.940435,5.636522
7496,This Time,Life of Agony,hardcore,5.104828,5.682414
7497,"Low Level (Listening, part II)",Stars of the Lid,ambient,4.965000,4.345000
7498,A Rainbow Aims,Brightblack Morning Light,indie,4.970000,3.570000


In [54]:
def to_alfanum(txt):
    return ''.join(ch for ch in txt if ch.isalnum() or ch == ' ')


lyrics = []
for i, (_, row) in enumerate(songsdata_df.iterrows()):
    track = to_alfanum(row['track'])
    author = to_alfanum(row['artist'])
    text = get_text_from_genius_of(author, track)
    lyrics.append(text)
    if i % 1000 == 0:
        print(i)

0


In [55]:
print(len([t for t in lyrics if t is None]))

11


In [56]:
songsdata_df['lyrics'] = lyrics

In [57]:
songsdata_df = songsdata_df.dropna()

In [58]:
songsdata_df.to_csv('lyrics1.csv')

''